In [ ]:
import glob
import datetime
import pandas as pd
import os
import re

# https://stackoverflow.com/questions/36587211/easiest-way-to-read-csv-files-with-multiprocessing-in-pandas
# http://python-3.ru/page/multiprocessing
from multiprocessing import Pool

In [ ]:
# Создаю/обнуляю список файлов 
files_full_path_list = list()

# Путь к корневому каталогу файлов
files_path = '/opt/app/data/shared/latest_dump/*/*.json'
# files_path = '/opt/app/data/shared/latest_dump/*/2018*.json'

In [ ]:
# Получаю перечень полных пути файлов в подкаталогах
for file_name in glob.glob(files_path, recursive=True):
    # Добавляю полный путь в список
    files_full_path_list.append(file_name)
    
files_full_path_list.sort()

print (datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'), 'Количество файлов:', len(files_full_path_list))

In [ ]:
files_full_path_list[:10]

In [ ]:
# os.path.getsize('/opt/app/data/shared/latest_dump/___top_links/2018-08-06.json')

In [ ]:
# !ls -all /opt/app/data/shared/latest_dump/___top_links/2018-08-06.json

In [ ]:
# %%time

# # CPU times: user 6min 38s, sys: 36.4 s, total: 7min 15s

# files_df = pd.DataFrame()

# for file in files_full_path_list:

#     file_size = os.path.getsize(file)
    
#     # re.findall('dump/(.+?)/\d', '/opt/app/data/shared/latest_dump/___top_links/2018-08-06.json')[0]
#     file_cat = re.findall('dump/(.+?)/\d', file)[0]
    
#     # re.findall('(\d\d\d\d-\d\d-\d\d)', '/opt/app/data/shared/latest_dump/___top_links/2018-08-06.json')[0]
#     file_date = re.findall('(\d\d\d\d-\d\d-\d\d)', file)[0]
    
#     list = [[file, file_size, file_cat, file_date]]
    
#     files_df = files_df.append(pd.DataFrame(list, columns=['file', 'file_size', 'file_cat', 'file_date']),ignore_index=True)

In [ ]:
files_df = pd.DataFrame()

def get_file_info(file):
    file_size = os.path.getsize(file)

    # re.findall('dump/(.+?)/\d', '/opt/app/data/shared/latest_dump/___top_links/2018-08-06.json')[0]
    file_cat = re.findall('dump/(.+?)/\d', file)[0]

    # re.findall('(\d\d\d\d-\d\d-\d\d)', '/opt/app/data/shared/latest_dump/___top_links/2018-08-06.json')[0]
    file_date = re.findall('(\d\d\d\d-\d\d-\d\d)', file)[0]

    list = [[file, file_size, file_cat, file_date]]
    
    return pd.DataFrame(list, columns=['file', 'file_size', 'file_cat', 'file_date'])

pool = Pool(processes=10)
df_list = pool.map(get_file_info, files_full_path_list)

files_df = pd.concat(df_list, ignore_index=True)

In [ ]:
files_df.tail()

In [ ]:
files_df.info()

In [ ]:
files_df['file_date'] = pd.to_datetime(files_df['file_date'], format='%Y-%m-%d')
files_df['file_size'] = files_df['file_size'].round(0).astype(int)

In [ ]:
files_df[['file_cat']]\
        .groupby(['file_cat'])['file_cat'] \
        .count() \
        .reset_index(name='count') \
        .sort_values(['count'], ascending=False) \
        .head(10)

In [ ]:
files_df[['file_cat', 'file_size']]\
        .groupby(['file_cat'])['file_size'] \
        .sum() \
        .reset_index(name='sum_kilobytes') \
        .sort_values(['sum_kilobytes'], ascending=False) \
        .head(10)

In [ ]:
import os
import pandas as pd 
from multiprocessing import Pool

# wrap your csv importer in a function that can be mapped
def read_csv(filename):
    'converts a filename to a pandas dataframe'
    return pd.read_csv(filename)


def main():
    # set up your pool
    pool = Pool(processes=8) # or whatever your hardware can support

    # get a list of file names
    files = os.listdir('.')
    file_list = [filename for filename in files if filename.split('.')[1]=='csv']

    # have your pool map the file names to dataframes
    df_list = pool.map(read_csv, file_list)

    # reduce the list of dataframes to a single dataframe
    combined_df = pd.concat(df_list, ignore_index=True)

if __name__ == '__main__':
    main()

In [ ]:
from multiprocessing import Pool

def doubler(number):
    return number * 2
 
numbers = [5, 10, 20]
pool = Pool(processes=3)
print(pool.map(doubler, numbers))

In [ ]:
# Сколько файлов по каталогам и какого размера каталоги?


In [ ]:
# Сколько служебных сообщений?
user leave channel 
user enter channel

In [ ]:
# Создаю пустой dataframe для данных их файлов
json_df = pd.DataFrame()

In [ ]:
%%time
# Наполняю данными о сообщениях за 2018 год dataframe (1min 31s) без multiprocessing
# Переделать на multiprocessing

for file in files_full_path_list:
    # Читаю файлы в dataframe
    data_parsed = json.loads(open(file).read())
    df = json_normalize(data_parsed)
    # Добавляю имя файла в dataframe для дальнейшего получения даты и названия канал
    df.insert(loc=0, column='FILE', value=file)
#     print (datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'), 'Добавляю содержимое файла в dataframe', file, round(os.path.getsize(file_name)/1000/1000,2), 'мегабайт')
    json_df = json_df.append(df, ignore_index=True, sort=False)

In [ ]:
# # Сохраняю dataframe в csv

# csv_file_name = '2018_ods_raw_new.csv'
# csv_file_dir = '/opt/app/data/'
# csv_file_path = csv_file_dir + csv_file_name

# # Проверка существует ли файл. Если существует удаляю
# if os.path.exists(csv_file_path):
#     os.remove(csv_file_name)
   
# print (datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'), 'Добавляю содержимое dataframe в csv', csv_file_path) 
# json_df.to_csv(csv_file_name, sep='|', index=False, encoding='utf-8')

# print (datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'), \
#        'Размер csv файла:', \
#        round(os.path.getsize(csv_file_path)/1000/1000,3), \
#        'Мегабайт')